<a href="https://colab.research.google.com/github/ritwin82/CodeBlue/blob/main/symptomchecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir my_fine_tuned_model

mkdir: cannot create directory ‘my_fine_tuned_model’: File exists


In [2]:
!pip install -q trl

In [4]:
!pip install -q -U transformers datasets accelerate peft trl bitsandbytes

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

In [9]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model_name = "Mistral-7B-Symptom-Checker"

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", # NormalFloat 4-bit quantization
    bnb_4bit_compute_dtype=torch.float16, # Use float16 for faster computation
    bnb_4bit_use_double_quant=False,
)

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1 # Recommended for Mistral

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [14]:
dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [15]:
peft_config = LoraConfig(
    r=16, # Rank of the update matrices
    lora_alpha=32, # LoRA scaling factor
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM", # This is for text generation
)

In [58]:
training_arguments = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=0.01, # Reduced number of training epochs for debugging
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    save_steps=100,
    logging_steps=100,
    gradient_checkpointing=True,
)

In [59]:
def formatting_func(example):
    # The dataset contains 'Description', 'Patient', and 'Doctor' columns.
    # We will format these into a conversational string for training.
    formatted_text = f"Description: {example.get('Description', '')}\nPatient: {example.get('Patient', '')}\nDoctor: {example.get('Doctor', '')}"
    return formatted_text


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_func,
    args=training_arguments,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [60]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,2.260100
200,2.216500
300,2.188900
400,2.141700
500,2.117100
600,2.110100


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

TrainOutput(global_step=643, training_loss=2.1686812500189845, metrics={'train_runtime': 3862.0234, 'train_samples_per_second': 0.665, 'train_steps_per_second': 0.166, 'total_flos': 3.028910171480064e+16, 'train_loss': 2.1686812500189845, 'entropy': 2.167377689550089, 'num_tokens': 709266.0, 'mean_token_accuracy': 0.5234743398289348, 'epoch': 0.010011054196702424})

In [62]:
trainer.save_model(new_model_name)